# Lab-3

In [27]:
def mod(number,module):

	numb_by_module=0

	if (number < 0):
	
		if (number // module != 0):
		
			numb_by_module = module - (abs(number) - module * (abs(number) // module))
		
		else:
		
			numb_by_module = module - abs(number)
		
	
	elif (number // module != 0):
	
		numb_by_module = number - module * (number // module)
	
	else:
	
		numb_by_module = number
	

	return numb_by_module



In [28]:
def gcd_extention( a, b): 
    q0=0
    r0=0
    r1=0
    r2=0
    u0=0
    u1=0
    u2=0

    if ((a == 0) and (b ==  0)): 
        u = v = 0
        return 0,u,v
    
    r2 = -a if (a < 0) else a

    r1 = -b if(b <  0) else b

    u2 = 1

    u1 = 0

    while (r1 != 0):
    
        q0 = r2 // r1
        r0 = r2 - q0 * r1
        r2 = r1
        r1 = r0
        u0 = u2 - q0 * u1
        u2 = u1
        u1 = u0
    
    u = u2

    if (a <  0):
        u = -(u)

    v =  0 if(b ==  0) else (r2 -(u) * a)//b

    return r2,u,v


In [29]:
def inverted_element(number,moduls):

	number = mod(number, moduls)

	gcd,u,v=gcd_extention(number, moduls)

	if (gcd ==1):
	
		b0 = moduls
		t=0
		q=0
		x0 = 0
		x1 = 1
		if (moduls == 1):
			return 1
		while (number > 1):
			q = number // moduls
			t = moduls
			moduls = number % moduls
			number = t
			t = x0
			x0 = x1 - q * x0
			x1 = t
		
		if (x1 < 0): x1 += b0

		return x1
	
	if (gcd > 1):
	
		return "Roots more or does`t exist!"
	


In [30]:
def chines_remainder_theorem(a,m):

    M = 1
    x = 0

    for i in range(0,len(m)):
        M *= m[i]
    
       
    for i in range(0,len(m)):

        n = M // m[i]
        u = inverted_element(n % m[i], m[i])
        
        x += n *u * a[i]
    

    x %= M

    if (x < 0):
        x += M
    
    general_module = M  

    return x,general_module


# Attack with small exponent applied on chines reminder theorem

In [31]:
e = 3

C1 =int("0x3115f665a5c62cfaeb9f3f0d2dfcfe8cafb4f90a005e20ea48d9b41607ef7188",base=16)
N1=int("0xEEB25A696A48E3DAAB70EC4C4BEF7C5998A07E465C90BD37F331F5BAA80011F9",base=16)
C2 =int("0x243f9d1059312b9daa01cae439cfdab7a4035364b04e5a993e43a68b79636b36",base=16)
N2 =int("0xAB5F12B623D023289CB3CAE70F1849808CE0C31F9733AD6F4AC2A5564DA84F2B",base=16)
C3 =int("0x20f6e6410982d39289cc4eacc04ea2ce8c853dece720f78e88963c5343c4659c",base=16)
N3 =int("0x9CBDBC7A89BB945021E1924C12A78122C0E0A7E8647AF5EDF9C47A9F021A5305",base=16)

In [32]:
x,general_module = chines_remainder_theorem([C1,C2,C3],[N1,N2,N3])

In [33]:
x

44125218104815898386247209166407765314394083745271945318076364289829068276115630296328267218157777265671054533549407754182863686419510155290410457059608290435142469249854696336135771044246717022207825658018396461132977

In [34]:
general_module

593313889622213826409958673492093976212725275872787606846018701036034457493748718491362759855713596132394686646895712927681554812581009969588944989289261752358334711132979307603323860886742956268321907131293547002663450714652914719

In [35]:
def diff(n, mid) :
    if (n > (mid * mid * mid)) :
        return (n - (mid * mid * mid))
    else :
        return ((mid * mid * mid) - n)

In [36]:
def cubicRoot(n) :
     
    # Set start and end for binary
    # search
    start = 0
    end = n
     
    # Set precision
    e = 0.0000001
    while (True) :
         
        mid = (start + end) / 2
        error = diff(n, mid)
 
        # If error is less than e
        # then mid is our answer
        # so return mid
        if (error <= e) :
            return mid
             
        # If mid*mid*mid is greater
        # than n set end = mid
        if ((mid * mid * mid) > n) :
            end = mid
             
        # If mid*mid*mid is less
        # than n set start = mid
        else :
            start = mid
             

print("Cubic root of", "is",
      hex(int(cubicRoot(x))))

Cubic root of is 0x2000000000000000000000000000000000000000000000000000000000000


In [37]:
import numpy as np
print(np.power(x,1/3))

3.5336941295567357e+72


In [38]:
# https://rosettacode.org/wiki/Nth_root#Python
from decimal import Decimal, getcontext

def nthroot (n, A, precision):
    getcontext().prec = precision
    
    n = Decimal(n)
    x_0 = A / n 
    x_1 = 1    
    while True:
        x_0, x_1 = x_1, (1 / n)*((n - 1)*x_0 + (A / (x_0 ** (n - 1))))
        if x_0 == x_1:
            return x_1

In [46]:
hex(int(nthroot (3, x, 100)))

'0x1ffffffffffffffff00633b0b2351cfcaa22b6539734270284c1d497c7891'

In [ ]:
C1 = 0x6ccdb4011f5db8ef3f29f142a650e8708081a9ceeaf9d353266224737d0ca36b1422b0a3e361b922a0f32ee501d263ee73396f672b5f03d299e154fc6f26df78d8f9829a1f58659d1e22ef9237d323e280a08bbda490a3b9e97bf989f3187a82229b993841a648743e319f5c7904bf3f7932457b9f2648e360eefbb6a85e6764
N1 = 0xC50813768871D20E4FC91709C56928DF050BF4324073021770782E986CBBC4CC43B1FCF73A9D1F53E612F1D44644AFDB2E7317FC7B8B89ABB4F1CFA3708AA8D69C6A5D5F0226A0D14DB36F4E5A7C3E5D233A04F106EE945910AD36CA69391EFAC4BB36FC4BE6A1DAAD4DB8A30B5E8514164FC92F71CAB6A8AEDF4B7FE52038EF
C2 = 0x432051a354b60a8dd0310fbab2977bf760b7d4798ec913472af399d44da2349ccd3403fada1a3054d4c0c4d87d77a2334af0c090e0a86b3c0d5e4e251af71682547181a924b32c082db8180a09a421670660fe79f4a4563a1801f2c3f8f908b69a0f30be7cf91142ec98ca91797d2ad258447b0a09086fc7f1014d08081f4c46
N2 = 0xC630C3A7E71523F4AB5421DB6B36B391839EDDF7EEEF2421ECA9C791D7A248F3A9AEDA9B8A71E64064E862AE01EAA6DBE8AE1D10A921D95CC678CE7F59A70CEDEC59D20489AECD8E5C835F0079C65A69B662C76FB7722DD4CD04B70593FA87E0916F313D4644F63DB56FFA9C76549DB23F3B32C558F8A64701A357899D6FE23F
C3 = 0x7504e178dc9f193844be5ab5598353a341e415081ab4b84646cec00793ad4ca2a2cc20b8420c52d5967bae13dd6d249ccf6cc92ee5ecf49514bf3b974d6e65f616e97bc9bebc06ff4b2c7d9922424a498bc0ffbad975cbbdd1fea3d782588966b7e9c96a2ed3176ec726f43ce3e4de8c2e651011bb89b9b31b00f69ea51c08aa
N3 = 0xB6C30E7E05A427FC564903718759D30F811CE6567E95DCBC166E296F28E5CB7BB5CFDE535C6F1E22215ECD465FC4668940828F1DAA2D1E7817BB766A7FA2A75D11118FB14D961DBC27448B280FA56312BA57848087F83D653BA64D6CB8168D5F077A51E0E72B370181A7102BC6C3406AB6733499C6E4F0A920BB3A0A1D7C1D19
C4 = 0x51aecd5864868da0f27513887a3efd1ce135519a4044afc6306d3dc40fa404f3aa41916ee71b4b2cddd08fd997f1da6a66a55280f23761cb19aef76ac3d15123b10f26b2f876e0914995990b4bd69537f472d7f6e13c6b7099cd74111655443aaaa60510536d51ffb976c576f7716e29baecbcad72eeb173dc230656c279749a
N4 = 0xB2ACD8D627C83C60FFE9F413727872735E40905361B1F0C820806C272D92CFDEF39D18D174EEDEA91B9B2CCD396F1946A8FA93B8D7A4BECA0827BCDF71A8D93AEF433C45DE0E0F5374099D174F990FC4A7AE1561441E1FF3DD0D36034C35139B05881FB08DDF20F49B71DFF024739951C20D021875E4C954040A26BBD1A998EB
C5 = 0x3dc5192d8c63fdfd3e6c97d3ade9e3d40314ca3b01b2b05f6f5b825b668107fa8d113d237db9776e5560b6421b17a881a417b7d9e74101bb52289b683fb5bc0b19f5300817ba9ccbd213eb57c1e5a169a7b180c3d5df3616d13f15618c0feca4509a6b86946e91d97ac2ed17f4d3ee8cf6b085034043fb10689f49345585cccf
N5 = 0xA79394E5ED5219FA8D701B8F8EEA2628D811FD7B087C08B73F4497D9B19E2039B61B4AF3FADDC7B2A11DB4818C03D5FD9084DAB6AEEB7172371EB2CC5B571E463A2ED98D2AE3BC5C5D41201C909EABBC377AFCA98E4347E8275CD19C29F1AD7757C9829906A104B5ACD1A3ADBBBE1E29D991303EADDD5DAA46A89CEA73DA3C39

# Attack-in-the-Middle